<a href="https://colab.research.google.com/github/Tachawla/sales_prediction/blob/main/Project1_Part5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

filename='/content/drive/MyDrive/sales_predictions.csv'
df=pd.read_csv(filename)
# 1. How many rows and columns?
df.shape
#8523 rows and 12 columns

#2.  What are the datatypes of each variable?
df.dtypes

#3.  Are there duplicates?  If yes, drop them.
df[df.duplicated(keep=False)]

#4 & #5 & #6  Identify missing values
df.isnull().sum()

# Missing values in Item_Weight and Outlet_Size
# The missing value rows are dropped since it seems
# difficult to interpolate values for the missing cells.
df_new=df.dropna()
df_new.isnull().any()

#7 Fix inconsistent data

#8 Summary statistics of numerical columns
df_new.describe()



,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,4650.000000,4650.000000,4650.000000,4650.000000,4650.000000
mean,12.898675,0.060700,141.716328,1999.190538,2272.037489
std,4.670973,0.044607,62.420534,7.388800,1497.964740
min,4.555000,0.000000,31.490000,1987.000000,69.243200
25%,8.770000,0.025968,94.409400,1997.000000,1125.202000
50%,12.650000,0.049655,142.979900,1999.000000,1939.808300
75%,17.000000,0.088736,186.614150,2004.000000,3111.616300
max,21.350000,0.188323,266.888400,2009.000000,10256.649000


In [54]:
# Project 1 Part 5 Begins.  Cleaned data from Part 4 above.
df_new.dtypes

# Item identifier, fat content, type, outlet identifier, outlet size, outlet 
# location type, outlet type

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [55]:
df_new = pd.get_dummies(df_new, columns = ['Item_Identifier', 'Item_Fat_Content', 'Item_Type',
'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type' ], drop_first = True)

In [51]:
df_new.dtypes

[dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype('uint8'),
 dtype

In [56]:
# imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [57]:
y = df_new['Item_Outlet_Sales']
X = df_new.drop(columns = 'Item_Outlet_Sales')

In [58]:
# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [59]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [63]:
print('LR Training RMSE:', np.sqrt(mean_squared_error(y_train, lr.predict(X_train))))
print('LR Testing RMSE:', np.sqrt(mean_squared_error(y_test, lr.predict(X_test))))

# The Testing RMSE is terrible.  Not a good model.

LR Training RMSE: 828.2652967498299
LR Testing RMSE: 2483870085354.562


In [62]:
print('Training R2:', lr.score(X_train, y_train))
print('Testing R2:', lr.score(X_test, y_test))

# The Testing R2 is typically between 0 and 1.  A negative R2 is puzzling.
# The linear model is probably not a good fit for this data.

Training R2: 0.6952017398168651
Testing R2: -2.7774889533738184e+18


In [67]:
# Question on features - unsure
